In [1]:
import csv
import cv2
import numpy as np
from keras import optimizers

lines = []
with open("driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
counter = 0
if(False):
    for line in lines:
        for i in range(3):
            sourcepath = line[i]
            filename = sourcepath.split('\\')[-1]
            currentpath = "IMG/" + filename
            image = cv2.imread(currentpath)
            images.append(image)
            measurements.append(float(line[3]))
            counter = counter + 1
        
for row in lines:
    steering_center = float(row[3])

    # create adjusted steering measurements for the side camera images
    correction = 0.2 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction

    # read in images from center, left and right cameras
    path = "..." # fill in the path to your training IMG directory
    img_center = cv2.imread("IMG/" + row[0].split('\\')[-1])
    img_left = cv2.imread("IMG/" + row[1].split('\\')[-1])
    img_right = cv2.imread("IMG/" + row[2].split('\\')[-1])

    # add images and angles to data set
    images.extend([img_center, img_left, img_right])
    measurements.extend([steering_center, steering_left, steering_right])
    
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement*-1.0)
    
x_train = np.array(images)
y_train = np.array(measurements)


from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: (x/255.0), input_shape = (160, 320, 3)))
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))

model.add(Conv2D(16, (5, 5), strides=(2, 3), activation="relu"))
model.add(Dropout(0.6))
#model.add(MaxPooling2D())
model.add(Conv2D(32, (5, 5), strides=(2, 3), activation="relu"))
model.add(Dropout(0.6))
model.add(Conv2D(64, (5, 5), strides=(1, 2), activation="relu"))
model.add(Dropout(0.4))
model.add(Conv2D(96, (5, 5), strides=(2, 2), activation="relu"))
model.add(Dropout(0.3))
model.add(Conv2D(128, (4, 4), strides=(1, 1), activation="relu"))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(520))
model.add(Dense(84))
model.add(Dense(1))

adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mse', optimizer=adam)
model.summary()

model.fit(x_train, y_train, validation_split=0.1, shuffle = True, epochs = 8)
model.save('model.h5')


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 43, 106, 16)       1216      
_________________________________________________________________
dropout_1 (Dropout)          (None, 43, 106, 16)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 34, 32)        12832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 34, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 15, 64)        51264     
__________